# Monte Carlo assessment

A Monte Carlo simulation is a way to assess uncertainties and robustness given probabilities on the model inputs. 

Here we illustrate how it works with a design Monte Carlo. It is also called robust design because we can  evaluate the impact on the designed characteristics (here the `exchanger.surface`).

In [1]:
from cosapp.drivers import MonteCarlo, NonLinearSolver, RunSingleCase
from cosapp.recorders import DataFrameRecorder
from cosapp.utils.distributions import Normal

from cpu.systems import CPUSystem

#### Start with a new system

In [2]:
cpu = CPUSystem("cpu")

#### Use a Monte Carlo driver to insert uncertainties on air temperature and observe impact on the system

In [3]:
# add MC driver
mc = cpu.add_driver(MonteCarlo("mc"))
mc.add_recorder(DataFrameRecorder(includes=["*"]))
mc.draws = 100

# Set the distribution of T_air around the current value
T_air = cpu.fan.inwards.get_details("T_air")
T_air.distribution = Normal(best=1, worst=-0.5)

# add T_air as perturbed variable
mc.add_random_variable("fan.T_air")

# check the impact on some response variables
mc.add_response(["fan.tension", "cpu.usage", "T_cpu"])

#### Add complementary drivers for design

In [4]:
design = mc.add_child(NonLinearSolver("design"))
design.extend(cpu.design_methods["exchanger_surface"])

runner = design.add_driver(RunSingleCase("runner"))
runner.set_values({"T_cpu": 80.0, "cpu.usage": 100.0})

#### Run Monte Carlo simulation 

In [5]:
cpu.run_drivers()
results = mc.recorder.export_data()

#### Show uncertainty distributions 

In [7]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4("Design of heat exchanger surface according to air and CPU temperatures"),
        dcc.Graph(id="graph"),
        dcc.Dropdown(
            id="response",
            options=[
                {"label": "Air temperature", "value": "fan.T_air"},
                {
                    "label": "Exchanger surface",
                    "value": "exchanger.surface",
                },
            ],
            value="fan.T_air",
        ),
    ]
)


@app.callback(Output("graph", "figure"), Input("response", "value"))
def display_color(response):
    fig = go.Figure(data=go.Histogram(x=results[response]))
    fig.update_layout(xaxis_title="Value", yaxis_title="Count")
    return fig


app.run(port=8051)